<a href="https://colab.research.google.com/github/JuanPabl07DP/SimulacionME/blob/main/Simulaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulación de bot para el monitoreo de emociones (Prototipo)

## Desarrollado por:
- Juan Pablo Daza Pereira
- Juan Sebastian Camargo Sanchez

## Subir archivos

In [ ]:
from google.colab import files

print("🔼 Sube una imagen (con rostro):")
uploaded_img = files.upload()

# Obtener el nombre del archivo
image_file = list(uploaded_img.keys())[0]

# Leer imagen como bytes
with open(image_file, 'rb') as image:
    image_bytes = image.read()

#print("🔼 Sube un audio (.wav o .mp3):")
#uploaded_audio = files.upload()

🔼 Sube una imagen (con rostro):


Saving Prueba2.jpg to Prueba2.jpg


## API Keys

In [ ]:
GOOGLE_VISION_API_KEY = ""
ASSEMBLYAI_API_KEY = ""
AZURE_FACE_API_KEY = ""
AZURE_FACE_API_ENDPOINT = ""
AWS_ACCESS_KEY_ID = ""
AWS_SECRET_ACCESS_KEY = ""
AWS_REGION = ""


## Envío de imagenes a Google Cloud Vision API

In [ ]:
import requests
import base64
import json

# Obtener nombre del archivo
image_file = list(uploaded_img.keys())[0]

# Codificar imagen a base64
with open(image_file, "rb") as image:
    img_base64 = base64.b64encode(image.read()).decode()

# Preparar solicitud
vision_url = f"https://vision.googleapis.com/v1/images:annotate?key={GOOGLE_VISION_API_KEY}"

payload = {
    "requests": [{
        "image": {
            "content": img_base64
        },
        "features": [{
            "type": "FACE_DETECTION",
            "maxResults": 1
        }]
    }]
}

response = requests.post(vision_url, json=payload)
face_data = response.json()

# Verifica si la respuesta contiene 'responses'
if "responses" in face_data:
    response_data = face_data["responses"][0]

    if "faceAnnotations" in response_data:
        emotions = response_data["faceAnnotations"][0]
        print("Emociones detectadas en la imagen:")
        for emotion in ['joyLikelihood', 'sorrowLikelihood', 'angerLikelihood', 'surpriseLikelihood']:
            print(f"{emotion}: {emotions[emotion]}")
    else:
        print("No se detectó ningún rostro en la imagen.")
else:
    print("Error en la respuesta de la API:")
    print(json.dumps(face_data, indent=2))


Emociones detectadas en la imagen:
joyLikelihood: VERY_LIKELY
sorrowLikelihood: VERY_UNLIKELY
angerLikelihood: VERY_UNLIKELY
surpriseLikelihood: VERY_UNLIKELY


## Envío de imagenes a Face API

In [ ]:
import requests
import json
# Ruta de la imagen subida en Colab
image_path = list(uploaded_img.keys())[0]

# Leer imagen en binario
with open(image_path, "rb") as image_file:
    image_data = image_file.read()

# Configurar headers y parámetros de Face API
headers = {
    "Ocp-Apim-Subscription-Key": AZURE_FACE_API_KEY,
    "Content-Type": "application/octet-stream"
}
params = {
    "returnFaceAttributes": "emotion"
}

# Hacer solicitud POST
face_api_url = f"{AZURE_FACE_API_ENDPOINT}/face/v1.0/detect"
response = requests.post(face_api_url, params=params, headers=headers, data=image_data)

# Procesar respuesta
if response.status_code == 200:
    results = response.json()
    if results:
        print("Emociones detectadas con Azure Face API:")
        emotions = results[0]["faceAttributes"]["emotion"]
        for emotion, value in emotions.items():
            print(f"{emotion.capitalize()}: {value:.3f}")
    else:
        print("No se detectaron rostros en la imagen.")
else:
    print("Error al consultar Azure Face API:")
    print(response.text)

Error al consultar Azure Face API:
{
  "error": {
    "code": "InvalidRequest",
    "message": "Invalid request has been sent.",
    "innererror": {
      "code": "UnsupportedFeature",
      "message": "Return Attributes (emotion, gender, age, smile, facial hair, hair and makeup) have been deprecated and are no longer supported. For more information, please see https://aka.ms/facerecognition."
    }
  }
}


## Envío de imagenes a Amazon Rekognition

In [ ]:
!pip install boto3

In [ ]:
import boto3
# Crear cliente de Rekognition
rekognition = boto3.client(
    'rekognition',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION
)

# Enviar solicitud para detectar emociones
response = rekognition.detect_faces(
    Image={'Bytes': image_bytes},
    Attributes=['ALL']
)

# Mostrar emociones
if 'FaceDetails' in response and len(response['FaceDetails']) > 0:
    print("Emociones detectadas con Amazon Rekognition:")
    emotions = response['FaceDetails'][0]['Emotions']
    for emotion in sorted(emotions, key=lambda x: x['Confidence'], reverse=True):
        print(f"{emotion['Type']}: {emotion['Confidence']:.2f}%")
else:
    print("⚠️ No se detectaron rostros en la imagen.")

Emociones detectadas con Amazon Rekognition:
HAPPY: 96.42%
CONFUSED: 0.44%
CALM: 0.28%
SURPRISED: 0.15%
SAD: 0.12%
DISGUSTED: 0.03%
ANGRY: 0.00%
FEAR: 0.00%


## Envío de audio a AssemblyAI

In [ ]:
import time

audio_file = list(uploaded_audio.keys())[0]

# Subir archivo a AssemblyAI
headers_auth = {'authorization': ASSEMBLYAI_API_KEY}
upload_url = "https://api.assemblyai.com/v2/upload"

with open(audio_file, 'rb') as f:
    upload_response = requests.post(upload_url, headers=headers_auth, files={'file': f})

audio_url = upload_response.json()['upload_url']

# Solicitar transcripción con análisis de emociones
transcribe_url = "https://api.assemblyai.com/v2/transcript"
payload = {
    "audio_url": audio_url,
    "speaker_labels": False,
    "iab_categories": True,
    "auto_highlights": True,
    "sentiment_analysis": True
}

transcribe_response = requests.post(transcribe_url, json=payload, headers=headers_auth)
transcript_id = transcribe_response.json()['id']

# Esperar a que termine
print("Analizando audio...")
status = "processing"
while status != "completed":
    polling = requests.get(f"https://api.assemblyai.com/v2/transcript/{transcript_id}", headers=headers_auth)
    result = polling.json()
    status = result["status"]
    time.sleep(3)

# Mostrar emociones
print("Emociones detectadas en el audio:")
for sentiment in result.get("sentiment_analysis_results", []):
    print(f"Texto: {sentiment['text']}")
    print(f"Emoción: {sentiment['sentiment']} | Confianza: {sentiment['confidence']:.2f}\n")

Analizando audio...
Emociones detectadas en el audio:
Texto: Historic ansato de todo histo channel.
Emoción: NEUTRAL | Confianza: 0.83

